# Random Forest

In [1]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform

In [ ]:
ctr_15 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_15.csv')
ctr_16 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_16.csv') 
data = pd.concat(([ctr_15,ctr_16]))
print("Tamaño del conjunto ", data.shape)
ctr_test = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv')
print("Tamaño del conjunto ", ctr_test.shape)

In [ ]:
# Lista de columnas a eliminar
columns_to_remove = [
    'device_id', 'action_list_1', 'action_list_2', 'action_categorical_0', 
    'auction_list_0', 'action_categorical_2', 'action_categorical_4', 
    'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_3', 
    'creative_categorical_0', 'auction_categorical_7', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6',
    'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10',
    'auction_categorical_4', 'gender', 'creative_categorical_1', 
    'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3',
    'auction_categorical_2', 'action_categorical_5', 'action_list_0',
    'auction_categorical_1', 'auction_categorical_5', 'device_id_type',
    'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0',
    'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5',
    'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8',
    'creative_categorical_4', 'creative_categorical_6', 'action_categorical_1',
    'creative_categorical_2', 'action_categorical_3', 'action_categorical_2',
    'auction_categorical_11', 'action_categorical_4',
    'creative_categorical_0', 'device_id'
]

data = data.drop(columns=columns_to_remove, errors='ignore')
ctr_test = ctr_test.drop(columns=columns_to_remove, errors='ignore')

id_col_test = ctr_test["id"]
ctr_test = ctr_test.drop(columns="id", errors='ignore')

print(f"Columnas eliminadas: {columns_to_remove}")
print(f"Tamaño del data después de la eliminación: {data.shape}")
print(f"Tamaño del test después de la eliminación: {ctr_test.shape}")


In [ ]:
def group_rare_categories(column, threshold):
    freq = column.value_counts(normalize=True)
    rare_categories = freq[freq < threshold].index
    return column.apply(lambda x: 'Other' if x in rare_categories else x)

threshold = 0.01

categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
categorical_columns_eval = ctr_test.select_dtypes(include=['object']).columns.tolist()

#categorical_columns = ['auction_categorical_12', 'auction_categorical_3', 'auction_categorical_7', 'auction_categorical_9']
#list_columns = ['action_list_2', 'action_list_1', 'auction_list_0']

for col in categorical_columns:
    data[col] = group_rare_categories(data[col], threshold)

for col in categorical_columns_eval:
    ctr_test[col] = group_rare_categories(ctr_test[col], threshold)
pd_ohe = pd.get_dummies(data,
                        columns=categorical_columns + ['has_video'],  # Incluye has_video también
                        sparse=False,    # Devolver una matriz no rala para facilitar la alineación
                        dummy_na=False,  # No agregar columna para NaNs.
                        dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )

pd_ohe_eval = pd.get_dummies(ctr_test,
                             columns=categorical_columns_eval + ['has_video'],  # Incluye has_video también
                             sparse=False,    # Devolver una matriz no rala
                             dummy_na=False,  # No agregar columna para NaNs.
                             dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                            )

pd_ohe_eval = pd_ohe_eval.reindex(columns=pd_ohe.columns, fill_value=0)

pd_ohe_eval['id'] = id_col_test.values

print("Shape de data_drop después de OneHotEncoding:", pd_ohe.shape)
print("Shape de eval_data_drop después de OneHotEncoding:", pd_ohe_eval.shape)

pd_ohe.info()
pd_ohe_eval.info()

In [ ]:
y = pd_ohe[['Label']].copy() # Usamos copy para no modificar un view abajo, ya que genera un warning.
X = pd_ohe.drop('Label', axis = 1)

X_train, X_val, Y_train, Y_val = train_test_split(X, y,
                                                  train_size = 0.8,
                                                  random_state = 32,
                                                  stratify = y)

X_test = pd_ohe_eval.drop('Label', axis = 1)

print(X_test.shape)
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')
